In [1]:
import os
os.chdir("../")
%pwd

'd:\\Bhavadeesh\\Education\\MLOps-Udemy\\projects\\data_science_project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: Path
    alpha: float
    l1_ratio: float
    target_column: str

In [7]:
from src.data_science.constants import *
from src.data_science.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
        schema_file_path: Path = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name,
        )
        return model_trainer_config

In [10]:
import pandas as pd
import os
from src.data_science import logger
from sklearn.linear_model import ElasticNet
import joblib


class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop(columns=[self.config.target_column], axis=1)
        X_test = test_data.drop(columns=[self.config.target_column], axis=1)
        y_train = train_data[self.config.target_column]
        y_test = test_data[self.config.target_column]

        model = ElasticNet(
            alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42
        )
        model.fit(X_train, y_train)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [11]:
try:
    confing = ConfigurationManager()
    model_trainer_config = confing.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-04-27 19:13:18,992: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-04-27 19:13:18,999: INFO: common: YAML file: params.yaml loaded successfully]
[2025-04-27 19:13:19,009: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-04-27 19:13:19,009: INFO: common: Directory created at: artifacts]
[2025-04-27 19:13:19,009: INFO: common: Directory created at: artifacts/model_trainer]
